In [ ]:
import os.path
import pandas as pd

import PyPDF2

from lingua import Language, LanguageDetectorBuilder
#languages = [Language.ENGLISH, Language.FRENCH, Language.GERMAN, Language.SPANISH]
#detector = LanguageDetectorBuilder.from_languages(*languages).build()
detector = LanguageDetectorBuilder.from_all_languages().with_preloaded_language_models().build()

In [ ]:
if (os.path.exists("data/langmeta.csv")):
    os.remove("data/langmeta.csv")

In [ ]:
pdfurl = pd.read_csv("data/downloadedpdf.csv").fillna(value = "")

In [ ]:
text= ""
for index, row in pdfurl.iterrows():
    
    unique = row['URL'].removeprefix("https://bildungsportal.sachsen.de/opal/oer/")
    unique = unique.removesuffix("\n")
    metadict = {"Identifier":unique}   
    pfad = "temp/" + unique + ".pdf"
    if((row['Error'] == "")):
        #Falls beim Download kein Error aufgetreten ist: Probier die Datei zu öffnen
        try:
            pdfFileObj = open(pfad, 'rb')
            pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
            for x in range(0,pdfReader.numPages):        
                pageObj = pdfReader.getPage(x)        
                text = text + pageObj.extractText()    
        except:
            print("An Error occured while reading.")
            #metadict.update({"Error":'OpeningError'})
            frame = pd.DataFrame(metadict, [0])
            if (os.path.exists("data/langmeta.csv")):   
                #oldframe = pd.read_csv("data/langmeta.csv")
                newframe = pd.concat([oldframe, frame])
                newframe.to_csv("data/langmeta.csv", index=False)
                oldframe = newframe
            else:
                frame.to_csv("data/langmeta.csv", index=False)
                oldframe = frame
            metadict.clear
            continue

    #Spracherkennung
        language = detector.detect_language_of(text) #compute_language_confidence_values(text)
        if(language is not None):
            metadict.update({"Language":language.name})
        frame = pd.DataFrame(metadict, [0])
        if (os.path.exists("data/langmeta.csv")):   
            #oldframe = pd.read_csv("data/langmeta.csv")
            newframe = pd.concat([oldframe, frame])
            newframe.to_csv("data/langmeta.csv", index=False)
            oldframe = newframe
        else:
            frame.to_csv("data/langmeta.csv", index=False)
            oldframe = frame
        metadict.clear
        text=""